# Runtime -> Change Runtime -> GPU

In [1]:
# check for the GPU provided in the runtime
!nvidia-smi
# using quiet method for controlling the log
# for suppressing the colored errors and warning in the terminal
!pip install --quiet transformers==4.1.1
# pytorch lightning for smoother model training and data loading
!pip install --quiet https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.2.6/pytorch-lightning-1.2.6.tar.gz 
# using HuggingFace tokenizers
!pip install --quiet tokenizers==0.9.4
# Google's sentencepiece
!pip install --quiet sentencepiece==0.1.94
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation
# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap
# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint
'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

Sun Apr 24 07:25:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Global seed set to 0


1.2.6
1.2.6


In [ ]:
# RuntimeError: CUDA out of memory. Tried to allocate 134.00 MiB (GPU 0; 15.90 GiB total capacity; 14.35 GiB already allocated; 29.75 MiB free; 14.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation. See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
# 이런 에러가 발생하는 이유는 batch size가 너무 크거나, 코드 상에서 메모리 누수가 발생했기 때문이라고 한다.
# 출처 : https://beausty23.tistory.com/145

import gc
gc.collect()
torch.cuda.empty_cache()

# Downloading the Dataset

In [ ]:
# # QA dataset from https://github.com/dmis-lab/bioasq-biobert
# # which is in Zip format
# !gdown --id 1mxVUywvKzvA9bvrUc11RYuOTy7MYcXHF
# # Unzipping the folder
# !unzip -q bio-QA.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1mxVUywvKzvA9bvrUc11RYuOTy7MYcXHF
To: /content/bio-QA.zip
100% 5.48M/5.48M [00:00<00:00, 327MB/s]


In [ ]:
# # let's have a look at one of the files
# with Path("BioASQ/BioASQ-train-factoid-4b.json").open() as json_file:
#   data_ = json.load(json_file)

In [ ]:
# # let's have a look at one of the files
# with Path("/content/drive/MyDrive/final_pjt/data/data.json").open() as json_file:
#   data = json.load(json_file)

KeyboardInterrupt: ignored

In [ ]:
# # let's have a look at one of the files
# with Path("/content/drive/MyDrive/final_pjt/data/KorQuAD_v1.0_train.json").open() as json_file:
#   data_ = json.load(json_file)

In [2]:
import os
import json
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
      data = json.load(f)
    data_rows = []
    for group in data['data']:
      for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          for answer in qa['answers']:
            answer_text = answer['text']
            answer_start = answer['answer_start']
            answer_end = answer['answer_start'] + len(answer_text)  #Gets the end index of each answer in the paragraph
            data_rows.append({
                  "question" : question,
                  "context"  : context,
                  "answer_text" : answer_text,
                  "answer_start" : answer_start,
                  "answer_end" : answer_end
              })

    return pd.DataFrame(data_rows)

df =read_squad('/content/drive/MyDrive/final_pjt/data/KorQuAD_v1.0_train.json')


In [3]:
df = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/ko1.0_add_ko2.0.csv')

In [ ]:
df1 = df[:50000]
df2 = df[50000:]
# df2 = df[70000:]

In [ ]:
df1.info()
len(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      50000 non-null  object
 1   context       50000 non-null  object
 2   answer_text   50000 non-null  object
 3   answer_start  50000 non-null  int64 
 4   answer_end    50000 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


50000

In [ ]:
df_t = df_t.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_t = df_t[['question', 'context', 'answer_text', 'answer_start', 'answer_end']]

In [ ]:
df_t.dropna(inplace = True)

In [ ]:
df_t = df_t.astype({'answer_start':int, 'answer_end':int})

In [ ]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12296 entries, 0 to 12305
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      12296 non-null  object
 1   context       12296 non-null  object
 2   answer_text   12296 non-null  object
 3   answer_start  12296 non-null  int64 
 4   answer_end    12296 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 576.4+ KB


In [ ]:
# Data is a dictionary
data.keys()
# Let’s have a look at how the data is stored and in what format.
Data['version']
# len of each file
len(data_['data'])
# We have a list of dictionaries in the "data". We can explore the 0th element
data['data'][0].keys()
data['data'][0]['title']
len(data['data'][0]['paragraphs'])
questions = data['data'][0]['paragraphs']
# datapoint sample
questions[0]

NameError: ignored

# Function to Create Pandas Dataframes of Questions and Answers
This function will help us read the data from the folder containing multiple JSON files and read them to a dataframe to run manipulation on it and proceed further. 

In [4]:
# def extract_questions_and_answers(factoid_path = Path):
#   with factoid_path.open() as json_file:
#     data = json.load(json_file)
#     questions = data['data'][0]['paragraphs']
#     data_rows = []
#     for question in questions:
#       context = question['context']
#       for question_and_answers in question['qas']:
#         question = question_and_answers['question']
#         answers = question_and_answers['answers']
#         for answer in answers:
#           answer_text = answer['text']
#           answer_start = answer['answer_start']
#           answer_end = answer['answer_start'] + len(answer_text)  #Gets the end index of each answer in the paragraph
#           data_rows.append({
#                 "question" : question,
#                 "context"  : context,
#                 "answer_text" : answer_text,
#                 "answer_start" : answer_start,
#                 "answer_end" : answer_end
#             })
#     return pd.DataFrame(data_rows)
# factoid_path = Path("BioASQ/BioASQ-train-factoid-4b.json")
# extract_questions_and_answers(factoid_path).head()      
# factoid_paths = sorted(list(Path('BioASQ/').glob('BioASQ-train-*')))
# factoid_paths
# dfs = []
# for factoid_path in factoid_paths:
#   df = extract_questions_and_answers(factoid_path)
#   dfs.append(df)
# df = pd.concat(dfs)
# df = df1
df.head()
df.shape
# Dropping all the rows with repeated context and questions pairs.
# df = df.drop_duplicates(subset=["context"]).reset_index(drop=True)
df.shape
len(df.question.unique())
len(df.context.unique())
sample_question = df.iloc[243]
sample_question
# Using textcolor to visualize the answer within the context
def color_answer(question):
  answer_start, answer_end = question["answer_start"],question["answer_end"]
  context = question['context']
  return  colored(context[:answer_start], "white") + \
    colored(context[answer_start:answer_end + 1], "green") + \
    colored(context[answer_end+1:], "white")
print(sample_question['question'])
print()
print("Answer: ")
for wrap in textwrap.wrap(color_answer(sample_question), width = 100):
  print(wrap)

악티늄족을 핵 연료로 이용할 때 사용해야 하는 방사능 차단 물질은 어떤 종류인가?

Answer: 
악티늄족을 핵 연료로 이용할 때는, 방사능원을 차단하거나 방사성 물질을 막을 수 있는 고급 광물(예로 들자면 큰 잠재적 이득이 있는 자체
발광 결정 등)을 사용해야 한다. 그러나, 악티늄족의 강한 방사능독으로 인해 자연에서의 이동과 악티늄족에 깊은 우려가 있기도 하다. 혼합산화물핵연료(MOX)에서 화학적으로 불안정한
형태를 가진 악티늄족의 사용과 방사능 원천을 차단하는 것으로는 현대 안전 기준에는 적절하지 않다. 현재 안정되고 내구성이 좋은 악티늄족과 관련된 물질을 제작하는 것에 도전하고
있다. 이것은 안전한 창고에 들어있어야 하고, 사용하고 나서 최종 처분할 수 있어야 한다. 악티늄족 고체 용액의 적용의 열쇠는 바로 내구성이 좋은 결정체의 상이다.


# Tokenization
In the following cells, we have instantiated the model and called its tokenizer. T5 tokenizer is pretty fast as compared to other BERT type tokenizers. We will run a sample of this on the text given below and do the decoding.

In [5]:
# using the base T5 model having 222M params
MODEL_NAME ='KETI-AIR/ke-t5-base' 
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
sample_encoding = tokenizer('is the glass half empty or half full?', 'It depends on the initial state of the glass. If the glass starts out empty and liquid is added until it is half full, it is half full. If the glass starts out full and liquid is removed until it is half empty, it is half empty.')
sample_encoding.keys()
print(sample_encoding["input_ids"])
print(sample_encoding["attention_mask"])
print(len(sample_encoding['input_ids']), len(sample_encoding['attention_mask']))
# Checking the decoding of the input ids
preds = [
        tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for input_id in sample_encoding['input_ids']
]
preds= " ".join(preds)
for wrap in textwrap.wrap(preds, width = 80):
  print(wrap)
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
encoding.keys()
tokenizer.special_tokens_map
tokenizer.eos_token, tokenizer.eos_token_id
# Input id of 1 represents end of sequence token.
# Text representation pf the input ids
tokenizer.decode(encoding['input_ids'].squeeze()) 

Downloading:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

[43, 5, 12857, 1648, 19876, 117, 1648, 1479, 85, 1, 223, 26000, 46, 5, 9113, 604, 14, 5, 12857, 3, 799, 5, 12857, 9162, 176, 19876, 13, 24483, 43, 1586, 1429, 60, 43, 1648, 1479, 4, 60, 43, 1648, 1479, 3, 799, 5, 12857, 9162, 176, 1479, 13, 24483, 43, 11514, 1429, 60, 43, 1648, 19876, 4, 60, 43, 1648, 19876, 3, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
63 63
is the glass half empty or half full ? </s> It depends on the initial state of
the glass . If the glass starts out empty and liquid is added until it is half
full , it is half full . If the glass starts out full and liquid is removed
until it is half empty , it is half empty . </s>


'악티늄족을 핵 연료로 이용할 때 사용해야 하는 방사능 차단 물질은 어떤 종류인가?</s> 악티늄족을 핵 연료로 이용할 때는, 방사능원을 차단하거나 방사성 물질을 막을 수 있는 고급 광물(예로 들자면 큰 잠재적 이득이 있는 자체 발광 결정 등)을 사용해야 한다. 그러나, 악티늄족의 강한 방사능독으로 인해 자연에서의 이동과 악티늄족에 깊은 우려가 있기도 하다. 혼합산화물핵연료(MOX)에서 화학적으로 불안정한 형태를 가진 악티늄족의 사용과 방사능 원천을 차단하는 것으로는 현대 안전 기준에는 적절하지 않다. 현재 안정되고 내구성이 좋은 악티늄족과 관련된 물질을 제작하는 것에 도전하고 있다. 이것은 안전한 창고에 들어있어야 하고, 사용하고 나서 최종 처분할 수 있어야 한다. 악티늄족 고체 용액의 적용의 열쇠는 바로 내구성이 좋은 결정체의 상이다.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

# Creating labels for the answers
In the following cells, We have to create necessary labels for the answers. This is required so that we can extract answers accordingly to the questions.

In [6]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
tokenizer.decode(answer_encoding['input_ids'].squeeze())
labels = answer_encoding["input_ids"]
labels
labels[labels == 0] = -100
labels 

tensor([[ 4928, 42481,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

# Create dataset
In the following cells, we have created the dataset for input in the model. This required setting the lengths, padding and tokenizer. The data has been taken for a BioQA dataset which is specifically for this task. 

In [7]:
class BioQADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    labels = target_encoding['input_ids']
    labels[labels==0] = -100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )
sample_dataset = BioQADataset(df, tokenizer)
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break 

Question:  바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
Answer text:  교향곡
Input_ids:  tensor([  396,   775, 21778, 63122,     6,  1649,  8643, 15171, 12022,  6937])
Labels:  tensor([58337,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])


# Split Dataset into Train and Test
In the following cells, we have split the data into two parts. Test size is small due to the heavy model needed large data for training.

In [8]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape,  val_df.shape 

((96849, 5), (5098, 5))

In [ ]:
train_df["answer_text"].str.len().max()

1886

# Create Pytorch Lightning Module
In the following cells, we have created the lightning module. This is made when we have complex models. Hence Pytorch has introduced this to smoothen the process.

In [9]:
 class BioDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer:T5Tokenizer,
       batch_size: int = 8,
       source_max_token_len: int = 396,
       target_max_token_len: int = 32,
       ):
     super().__init__()
     self.train_df = train_df
     self.test_df = test_df
     self.tokenizer = tokenizer
     self.batch_size = batch_size
     self.source_max_token_len = source_max_token_len
     self.target_max_token_len = target_max_token_len
   def setup(self):
     self.train_dataset = BioQADataset(
         self.train_df,
         self.tokenizer,
         self.source_max_token_len,
         self.target_max_token_len
         )
     self.test_dataset = BioQADataset(
     self.test_df,
     self.tokenizer,
     self.source_max_token_len,
     self.target_max_token_len
     )
   def train_dataloader(self):
     return DataLoader(
         self.train_dataset,
         batch_size=self.batch_size,
         shuffle=True,
         num_workers=4
         )
   def val_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=self.batch_size,
         num_workers=4
         )
   def test_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=1,
         num_workers=4
         )
 BATCH_SIZE = 4
 N_EPOCHS = 6
 data_module = BioDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
 data_module.setup() 

# Loading and Fine Tuning T5
In the following cells, we have installed the model and fine tuned it to our application requirements. 

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)
# model.config
# To check the translation from English to German built-in task 
# input_ids_translated = tokenizer(
#     "translate English to German : Oppertunity did not knock until I built a door",
#     return_tensors = 'pt'
# ).input_ids
# generated_ids = model.generate(input_ids = input_ids_translated)
# generated_ids
# pred_translated = [
#         tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         for gen_id in generated_ids
# ]
# pred_translated
# text = "".join(pred_translated)
# input_ids_summary = tokenizer(
#     text,
#     return_tensors = 'pt'
# ).input_ids
# generated_ids_summary = model.generate(input_ids = input_ids_summary)
# generated_ids_summary
# pred_summary = [
#         tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         for gen_id in generated_ids
# ]
# " ".join(pred_summary)
# Model config
model.config

Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

T5Config {
  "_name_or_path": "KETI-AIR/ke-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.4.2",
  "use_cache": true,
  "vocab_size": 64128
}

# Building the PyTorch lightning module using T5ForConditionalGeneration model
In the following cells, we have leveraged a T5 Conditional Generator which will produce text based on some condition (this is going to be our application task – QA).

In [10]:
class BioQAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)
    return output.loss, output.logits
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}
  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer
model = BioQAModel()

Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

# Using Trainer from PyTorch-Lightning to Finetune Model Using our Dataset
We have used a trainer module to test the model and fine-tune it in the following cells. This is helpful when the model is huge, and you want to get a warmup of the model without changing the learning rate. 

In [ ]:
model = BioQAModel()
# To record the best performing model using checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/final_pjt/ET5/model",
    filename="best-checkpoint-large",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)
#logger = TensorBoardLogger("training-logs", name="bio-qa")
#logger = TensorBoardLogger("training-logs", name="bio-qa")
trainer = pl.Trainer(
    #logger = logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
) 

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Checkpoint directory /content/drive/MyDrive/final_pjt/ET5/model exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# Loading Tensorboard
In the following cells, we have used a Tensorboard for seeing how the model is progressing against time and epochs. This is very helpful when running huge models. 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs
#!rm --rf lightning_logs
trainer.fit(model, data_module)
trainer.test()  # evaluate the model according to the last checkpoint

# Predictions

In [394]:
trained_model = BioQAModel.load_from_checkpoint("/content/drive/MyDrive/final_pjt/ET5/model/best-checkpoint-large-v1.ckpt")
# trained_model = BioQAModel.load_from_checkpoint('/content/drive/MyDrive/final_pjt/ET5/model/best-checkpoint.ckpt')
trained_model.freeze() # 

In [ ]:
# f1 score
from sklearn.metrics import f1_score

pred = trained_model.predict(val_df)

# Generate Answers for the Questions in the Validation Set
In the following cells, we have tried and tested some questions whose answers were saved with us in the valid dataset against the predicted values given by the model. As you’ll notice the T5 model is so powerful that it will predict exact values.

In [136]:
# 키워드 추출

def key_generate_answer(question):
  source_encoding=tokenizer(
      question,
      key_test(input('키워드 : ')),
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [137]:
key_generate_answer(input('질문 : '))

질문 : 남한산성을 훼손시킨 것은 누구인가?
키워드 : 남한산성


'김훈'

In [108]:
# 질문 토큰화 - test 함수 사용

def generate_answer(question):
  source_encoding=tokenizer(
      question,
      test(question),
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [135]:
# sample_question = val_df.iloc[20]
# sample_question["question"]
# sample_question["answer_text"]  # Label Answer
# generate_answer(sample_question)  # Predicted answer
# sample_question = val_df.iloc[7]
# sample_question["answer_text"]
generate_answer(input('질문 : '))

KeyboardInterrupt: ignored

In [110]:
# 질문 토큰화 - val_test 함수 사용

def val_generate_answer(question):
  source_encoding=tokenizer(
      question,
      val_test(question),
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [115]:
val_generate_answer(input('질문 : '))

질문 : 남한산성을 훼손시킨 것은 누구인가?


'이자영의 보고'

In [452]:
# 질문 토큰화 - cos유사도 추가

def cos_generate_answer(question):
  source_encoding=tokenizer(
      question,
      cos_test(question),
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [447]:
cos_generate_answer(input('질문 : '))

질문 : ㅇ


'훈민정음의 자음자'

In [453]:
for i in samp_q:
  print(cos_generate_answer(i))

왕립독립기념서
2010년
7850점
데이터
4개
왕즈 매니지먼트
2006년
왕립 핵무기 개발 보상금
황희환
제2의 세계


# 사용자 질문 받으면 올바른 context에 접근하게 하는 알고리즘 만들기

### 데이터 로드

In [13]:
import pandas as pd

test_df = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/data_add_title.csv')

In [449]:
samp = val_df.sample(10, ignore_index = True)
samp_q, samp_a = list(samp['question']), list(samp['answer_text'])

In [450]:
samp_q

['여운형이 한국독립에 관한 청원서를 작성한 해는?',
 '알파고 시나씨가 지한통신사 특파원으로 활동하기 시작한 것은 언제일까?',
 '세계 피겨 스케이팅 선수권대회에서 쇼트프로그램 몇점으로 선두에 올랐나?',
 '원격 의료의 단점으로 전송된 기록의 상태가 좋지 않을 시, 문제가 발생할 수 있는 부분은?',
 '2014년에 작성된 북한에 대한 UPR중 북한이 거부한 권고안 수는?',
 '댄스짱 오디션을 통해 정윤호를 발탁한 매니지먼트는? ',
 '한기주 선수가 기아타이거즈에 입단한 해는?',
 '핵무기 개발을 위해 선정된 지역의 이주민들에게 1945년 3월까지 토지보상금으로 지급된 금액은?',
 '오차드의 위증 자백서를 이용하여 WFM 지도부에 누명을 씌운 사람은 누구인가?',
 '수필 요정 이야기에 대해서 제 2의 세계라고 한 인물은?']

In [451]:
samp_a

['1918년',
 '2010년',
 '69.97',
 '환자 케어 정보',
 '83개',
 'SM엔터테인먼트',
 '2006년',
 '260만 달러',
 '맥파를랜드',
 '톨킨']

In [70]:
val_df = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/val_df.csv')

In [73]:
val_df.sample(10, ignore_index = True)

,question,context,answer_text,answer_start,answer_end
0,리베리오가 사망한 때는 언제인가?,366년 9월 24일 리베리오가 선종하자 로마 교회는 두 파벌로 분열되었다. 하나는...,366년 9월 24일,0,11
1,아우구스투스가 창설한 황제의 친위대는?,아우구스투스의 통치는 로마의 평화라 불리는 태평성대를 이루었다. 계속되는 변방에서의...,로마 근위대,243,249
2,괴벨스가 예술비평에 대해 반대를 표명한 해는 언제인가?,안톤 브루크너 안톤 브루크너 브루크너는 여기로 연결됩니다. 다른 뜻에 대해...,1936년,17009,17014
3,서울에서 핵안보정상회의가 열린 것은 몇 년도입니까?,2012년 핵안보정상회의 2012년 핵안보정상회의 2012 서울 핵안보정상...,2012,418,422
4,암살단파가 명성을 얻기 시작한 것은 이란계 지도자가 파티마조와 결별하고 이란과 시리...,암살단파(Hashshashin)는 '산상의 노인'이라 불렸던 이란계 지도자가 11세...,요새,79,81
5,사나칸이 세이프 가드의 모습일 때 자아는 무엇에 의해 통제되는가?,"만화상 짧게 등장하는 단발머리 소녀의 모습의 상위 세이프 가드로, 키리이를 세이프 ...",중앙 세이프 가드,368,377
6,사랑은 뭐길래의 방송연도는?,1997년 CCTV 채널1에서 방영된 드라마 《사랑은 뭐길래》 이후 일어난 한류 열...,1997년,0,5
7,2017년 12월 개봉한 영화 <1987>에서 유일한 허구 인물인 87학번 신입생 ...,2017년 12월 박종철 고문치사 사건부터 6월 항쟁까지 이어지는 한국 현대사를 담...,김태리,80,83
8,메건 폭스가 오른쪽 엄지 손가락에 앓고 있는 질환은?,폭스는 또한 오른손 엄지 손가락이 단지증을 앓고 있다고한다. 2010년에는 강박 장...,단지증,19,22
9,닉 제닝스가 맡은 역할은 무엇인가?,곧 워드와 그의 제작 팀은 스토리보드와 각본 제작을 시작하였다. 그러나 카툰 네트워...,미술 감독,423,428


## 1. 키워드 문자 받기

In [174]:
# 키워드 문자 받기

def key_test(keyword):
  key_df = test_df.query(f'title.str.contains("{keyword}")', engine = 'python')
  # key_context = key_df.query(f'context.str.contains("{question}")', engine = 'python')
  use_context = key_df.drop_duplicates(['context'], ignore_index = True)

  context_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
  
  context = " ".join(context_li)

  return context_li[1]

In [170]:
key_test(input('키워드 : '))

키워드 : 남한산성


'남한산성 은 2017년 10월 3일에 개봉한 대한민국의 영화이다 황동혁 감독이 소설가 김훈의 동명 소설을 영화화한 것으로 병자호란 당시 남한산성에 갇힌 인조가 청나라에 항복하기까지의 과정을 그리며 정치에 대해 메시지를 던진다 1636년 병자호란 당시 남한산성에 피난하게 된 인조는 사방이 청군에 포위되어 고립무원의 처지에 놓인다 남한산성에는 자원도 풍부하지 못해 50일가량 견딜 수 있다 청에 저항하는 50일 남짓 대신들의 의견도 첨예하게 갈린다 이조판서 최명길 이병헌 은 치욕을 당하더라도 청에게 항복해 백성을 지켜야 한다고 주장했고 예조판서 김상헌 김윤석 은 청에 끝까지 맞서 싸워야 한다고 주장한다 그 속에서 청군의 압박은 점점 심해진다 남한산성에 갇혀 결국 청에 항복하기까지의 47일 동안 주화파와 척화파의 대립 속에서 번민하는 임금과 저마다의 충 忠 을 따르고자 하는 신하들 전쟁으로 고통받는 백성들 등 다양한 인간군상이 묘사된다 2016년 11월 제작이 시작되어 2017년 4월까지 촬영되었다 제작비는 총 150억 원 정도가 사용되었다 황동혁 감독은 소설에 충실한 영화를 만들기 위해 액션사극이나 퓨전사극 풍을 배제하고 정통사극의 분위기로 인물과 이야기에 집중했으며 화면 구성과 음악에서도 레버넌트 죽음에서 돌아온 자와 같이 절제된 느낌을 내려했다고 말한다 영화를 통해 인물들과 그들의 간절함 민초들의 생명력과 노력을 보여주고 싶었다면서 그래서 이 영화의 결말이 인조의 치욕스러운 모습이 아닌 서날쇠의 힘찬 망치소리와 봄의 풍경 속으로 뛰어드는 아이 나루의 모습이었던 거죠 라고 말했다 남한산성은 대한민국에서 2017년 10월 3일에 개봉했다 추석 연휴 기간과 맞물려 개봉 이틀만에 역대 최단기간으로 100만 관객을 넘었으며 개봉 후 4일만에 200만 관객을 돌파했다 그러나 추석 연휴 이후 관객수 증가 추세가 감소하여 영화의 손익분기점인 500만 관객에도 미치지 못하면서 상업적으로는 실패했다는 평가를 받는다 남한산성은 대체로 긍정적인 평가를 받았다 이동진 평론가는 5점 

## 2-1. 질문 토큰화(cos유사도 X)

- 토큰화 함수 불러오기

In [18]:
# !pip install JPype1-0.63-cp36-cp36m-xin_amd64.whl
!pip install konlpy

     |████████████████████████████████| 19.4 MB 4.7 MB/s 
     |████████████████████████████████| 448 kB 48.0 MB/s 


In [19]:
from konlpy.tag import Okt
okt = Okt()

In [252]:
token = okt.phrases(input('질문 : '))
# token = okt.nouns(input('질문 : '))

질문 : 남한산성을 훼손시킨 것은 누구인가?


In [261]:
token

['남한산성', '훼손', '훼손시킨 것', '누구', '남한', '산성']

In [119]:
token_len = len(token) - (len(token)-2)
token = token[:token_len]
token

['남한산성', '훼손']

In [275]:
test_df.query(f'title.str.contains("{token_}")', engine = 'python')

,question,context,answer_text,answer_start,answer_end,title
943,다모 산성에 있는 천수 또는 망루는 몇층이야?,다몬 산성 일본어 多聞山城 다몬야마조 은 나라현 나라 시 호렌 정에 있던 평산성이다...,4층,2133,2135,다몬_산성
944,혼마루는 몇m의 길이를 가지고 있는가?,다몬 산성 일본어 多聞山城 다몬야마조 은 나라현 나라 시 호렌 정에 있던 평산성이다...,140m,10176,10180,다몬_산성
1802,일본 농림수산성의 부서 중 동물의 위생을 관리하는 부서의 명칭은?,농림수산성 일본어 農林水産省 노린스이산쇼 the ministry of agricul...,동물위생과,5672,5686,일본_농림수산성
1999,남해의 대국산성은 어디에 위치해 있을까?,남해 대국산성 南海 大局山城 은 경상남도 남해군 설천면 진목리 대국산의 정상에 지어...,경상남도 남해군 설천면 진목리 대국산의 정상에 지어진 산성,2091,2123,남해_대국산성
2164,부소산성은 어느 시대의 도성인가?,부소산성 扶蘇山城 은 백제의 마지막 왕성으로 백제 시대에는 사비성 泗沘城 이라 불렸...,사비시대,2667,2671,부소산성
...,...,...,...,...,...,...
77114,장경사에서 가장 화려한 고건축 양식으로 지어진 건물은?,세계유산목록에 따른 정식명칭 유네스코에 의해 구분된 지역 남한산성 南漢山城 은 대한...,대웅전,42044,42047,남한산성
77115,남한산성 안에 있던 9개의 사찰 중 당시의 모습으로 현존하고 있는 유일한 사찰의 이름은?,세계유산목록에 따른 정식명칭 유네스코에 의해 구분된 지역 남한산성 南漢山城 은 대한...,장경사,41730,41733,남한산성
77116,병자호란 당시 남한산성이 함몰 위기에 놓였을때 어영별장으로 있던 자의 이름은?,세계유산목록에 따른 정식명칭 유네스코에 의해 구분된 지역 남한산성 南漢山城 은 대한...,이기축,41959,41962,남한산성
77117,남한산성 내에 존재했던 9개의 사찰 가운데 당시의 모습으로 남아있는 현존하는 유일한...,세계유산목록에 따른 정식명칭 유네스코에 의해 구분된 지역 남한산성 南漢山城 은 대한...,장경사,41730,41733,남한산성


In [273]:
text = ''
for i in range(len(token)):
  text += f'{token[i]}|'

token_ = text[:-1]

### 1) test_df 기반 코드(title - 있음)

In [97]:
# test_df 기반 코드(title - 있음)

def test(user_question):
  token_li = okt.phrases(user_question)
  token_len = len(token_li) - (len(token_li)-2)
  token = token_li[:token_len]

  key_title = test_df.query(f'title.str.contains("{token[0]}|{token[1]}")', engine = 'python')
  if len(key_title) == 0:
    key_question = test_df.query(f'question.str.contains("{token[0]}|{token[1]}")', engine = 'python')
    if len(key_question) == 0:
     key_context = test_df.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
    else:
     key_context = key_question.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
     if len(key_context) == 0:
      use_context = key_question.drop_duplicates(['context'], ignore_index = True)
     else:
      use_context = key_context.drop_duplicates(['context'], ignore_index = True)

  else:
    key_question = key_title.query(f'question.str.contains("{token[0]}|{token[1]}")', engine = 'python')
    if len(key_question) == 0:
      key_context = key_title.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
    else:
      key_context = key_question.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
      if len(key_context) == 0:
        use_context = key_question.drop_duplicates(['context'], ignore_index = True)
      else:
        use_context = key_context.drop_duplicates(['context'], ignore_index = True)


  context_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
  
  context = " ".join(context_li)

  return context

In [ ]:
# 바그너가 처음 교향곡을 작곡한 장소는?
test(input('질문 : '))

### 2) val_df 기반 코드(title - 없음)

In [95]:
# val_df 기반 코드(title - 없음)

def val_test(user_question):
  token_li = okt.phrases(user_question)
  token_len = len(token_li) - (len(token_li)-2)
  token = token_li[:token_len]
  
  key_question = val_df.query(f'question.str.contains("{token[0]}|{token[1]}")', engine = 'python')
  if len(key_question) == 0:
    key_context = val_df.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
  else:
    key_context = key_question.query(f'context.str.contains("{token[0]}|{token[1]}")', engine = 'python')
    if len(key_context) == 0:
      use_context = key_question.drop_duplicates(['context'], ignore_index = True)
    else:
      use_context = key_context.drop_duplicates(['context'], ignore_index = True)


  context_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
  
  context = " ".join(context_li)

  return context

In [117]:
# 바그너가 처음 교향곡을 작곡한 장소는?
val_test(input('질문 : '))

질문 : 남한산성을 훼손시킨 것은 누구인가?


'전략 거점으로서 남한산성의 중요성은 조선 태종 때 본격화되었다. 1401년 요동을 다녀온 이자영의 보고에 의거, 명나라가 조선을 침략할 지도 모른다는 위기감이 고조되자 태종은 국경과 내륙을 막론하고 고을 별로 산성을 쌓고 기존의 산성을 개축하여 산성 중심의 방어체제를 구축하기 시작했다. 남한산성도 이러한 국방정책에 따라 세종 때에 군사시설로 이용하기 시작했다. 아직 태종 때부터 세종 때까지 남한산성이 개축되었다는 기록은 발견되지 않았으나, 《세종실록지리지》에서 남한산성에 대해 비교적 상세하게 현황을 파악한 기록이 있는데, 이 기록은 남한산성을 이용하기 위해 현황을 파악한 것으로 볼 수 있으며, 따라서 이를 통해 당시 개축이나 수리는 안했어도 군사기지로 이용하기 시작했음을 알 수 있다. 2006 년 1 월 1 일 호주에서 명예 훼손 혐의로 고소 될 수있는 권리를 엄격히 제한하는 호주의 통일적인 명예 훼손 법 개혁이 있었다. (명예 훼손 법령 Defamation Act 2005 (빅), 9 페이지 참조). 이로 인해 주 및 테리토리의 명예 훼손 관련 법이 비슷하다. 명예 훼손의 일반 금지 규정에서 제외 된 유일한 기업은 비영리 법인 또는 직원 수가 10 명 미만이며, 다른 회사와 제휴하지 않은 비영리 법인 이다. 그러나 원고는 명예 훼손이 악의로 만들어졌으며, 경제적 손실을 초래했음을 입증해야 명예훼손 죄로 고소가 가능하다, 또한, 회사는 여전히 명예 훼손보다 더 큰 중상모략을 통해 입은 거짓 소송에 대해는 명예 훼손으로 고소 할 수 있다.'

## 2-2. 질문 토큰화(cos유사도 O)

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer

sent=(key_test(input('키워드 : ')),
      "남한산성을 훼손시킨 것은 누구인가?")
      
tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(sent)

idf=tfidf_vectorizer.idf_

print(dict(zip(tfidf_vectorizer.get_feature_names(),idf)))

키워드 : 남한산성
{'076의': 1.4054651081081644, '08m': 1.4054651081081644, '10개가': 1.4054651081081644, '10개의': 1.4054651081081644, '10년': 1.4054651081081644, '10년에': 1.4054651081081644, '119호': 1.4054651081081644, '11개': 1.4054651081081644, '11암문이': 1.4054651081081644, '11월': 1.4054651081081644, '11칸': 1.4054651081081644, '1231년과': 1.4054651081081644, '1232년': 1.4054651081081644, '124결이라는': 1.4054651081081644, '125개소의': 1.4054651081081644, '12年': 1.4054651081081644, '12년': 1.4054651081081644, '12년에': 1.4054651081081644, '12년에는': 1.4054651081081644, '12를': 1.4054651081081644, '12만명의': 1.4054651081081644, '12목을': 1.4054651081081644, '12월': 1.4054651081081644, '12주': 1.4054651081081644, '1300mm': 1.4054651081081644, '137종': 1.4054651081081644, '13m': 1.4054651081081644, '13년': 1.4054651081081644, '13층': 1.4054651081081644, '1400mm이다': 1.4054651081081644, '1401년': 1.4054651081081644, '1402명의': 1.4054651081081644, '140368평': 1.4054651081081644, '140368평이': 1.4054651081081644, '1410': 1.405465108108

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [183]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0]

0.049571650503251706

In [185]:
dd = [1, 2, 3, 0.049571650503251706]
dd.index(min(dd))

3

In [387]:
# cos 유사도 추가

def cos_test(user_question):
  token_li = okt.phrases(user_question)
  token_len = len(token_li) - (len(token_li)-3)
  token = token_li[:token_len]

  key_title = test_df.query(f'title.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
  if len(key_title) == 0:
    key_question = test_df.query(f'question.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
    if len(key_question) == 0:
     key_context = test_df.query(f'context.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
    else:
     key_context = key_question.query(f'context.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
     if len(key_context) == 0:
      use_context = key_question.drop_duplicates(['context'], ignore_index = True)
     else:
      use_context = key_context.drop_duplicates(['context'], ignore_index = True)

  else:
    key_question = key_title.query(f'question.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
    if len(key_question) == 0:
      key_context = key_title.query(f'context.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
    else:
      key_context = key_question.query(f'context.str.contains("{token[0]}|{token[1]}|{token[2]}")', engine = 'python')
      if len(key_context) == 0:
        use_context = key_question.drop_duplicates(['context'], ignore_index = True)
      else:
        use_context = key_context.drop_duplicates(['context'], ignore_index = True)


  context_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
  
  # context = " ".join(context_li)

  # cos 유사도가 가장 높은 context 추출
  tfidf_vectorizer=TfidfVectorizer()
  cos = []

  for i in range(len(context_li)):
    sent=(context_li[i],
          user_question)
    tfidf_matrix=tfidf_vectorizer.fit_transform(sent)
    cos.append(cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0])

  context_idx = cos.index(max(cos))

  return context_li[context_idx]
  # return max(cos)

In [401]:
# cos 유사도 추가 - 모든 단어 넣기(OR 연산)

def cos_test(user_question):
  token_li = okt.phrases(user_question)
  # token_len = len(token_li) - (len(token_li)-3)
  # token = token_li[:token_len]
  text = ''
  for i in range(len(token_li)):
    text += f'{token_li[i]}|'

  token = text[:-1]

  key_title = test_df.query(f'title.str.contains("{token}")', engine = 'python')
  if len(key_title) == 0:
    key_question = test_df.query(f'question.str.contains("{token}")', engine = 'python')
    if len(key_question) == 0:
     key_context = test_df.query(f'context.str.contains("{token}")', engine = 'python')
    else:
     key_context = key_question.query(f'context.str.contains("{token}")', engine = 'python')
     if len(key_context) == 0:
      use_context = key_question.drop_duplicates(['context'], ignore_index = True)
     else:
      use_context = key_context.drop_duplicates(['context'], ignore_index = True)

  else:
    key_question = key_title.query(f'question.str.contains("{token}")', engine = 'python')
    if len(key_question) == 0:
      key_context = key_title.query(f'context.str.contains("{token}")', engine = 'python')
    else:
      key_context = key_question.query(f'context.str.contains("{token}")', engine = 'python')
      if len(key_context) == 0:
        use_context = key_question.drop_duplicates(['context'], ignore_index = True)
      else:
        use_context = key_context.drop_duplicates(['context'], ignore_index = True)


  context_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
  
  # context = " ".join(context_li)

  tfidf_vectorizer=TfidfVectorizer()
  cos = []

  for i in range(len(context_li)):
    sent=(context_li[i],
          user_question)
    tfidf_matrix=tfidf_vectorizer.fit_transform(sent)
    cos.append(cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0])

  context_idx = cos.index(max(cos))

  return context_li[context_idx]
  # return max(cos)

In [ ]:
cos_test(input('질문 : '))

In [410]:
# cos 유사도 추가 - 모든 단어 넣기(AND 연산)

def cos_test(user_question):
  token_li = okt.phrases(user_question)
  # token_len = len(token_li) - (len(token_li)-3)
  # token = token_li[:token_len]
  text = ''
  for i in range(len(token_li)):
    text += f'{token_li[i]}|'

  token = text[:-1]

  try:
    key_title = test_df[test_df['title'].map(lambda x: all(string in x for string in token_li))]
    if len(key_title) == 0:
      key_question = test_df[test_df['question'].map(lambda x: all(string in x for string in token_li))]
      if len(key_question) == 0:
       key_context = test_df[test_df['context'].map(lambda x: all(string in x for string in token_li))]
      else:
        key_context = key_question[key_question['context'].map(lambda x: all(string in x for string in token_li))]
        if len(key_context) == 0:
          use_context = key_question.drop_duplicates(['context'], ignore_index = True)
        else:
          use_context = key_context.drop_duplicates(['context'], ignore_index = True)

    else:
      key_question = key_title[key_title['question'].map(lambda x: all(string in x for string in token_li))]
      if len(key_question) == 0:
        key_context = key_title[key_title['context'].map(lambda x: all(string in x for string in token_li))]
      else:
        key_context = key_question[key_question['context'].map(lambda x: all(string in x for string in token_li))]
        if len(key_context) == 0:
          use_context = key_question.drop_duplicates(['context'], ignore_index = True)
        else:
          use_context = key_context.drop_duplicates(['context'], ignore_index = True)
    

    context_li = []
    for i in range(len(use_context)):
      context_li.append(use_context['context'][i])
    
    # context = " ".join(context_li)

    tfidf_vectorizer=TfidfVectorizer()
    cos = []

    for i in range(len(context_li)):
      sent=(context_li[i],
            user_question)
      tfidf_matrix=tfidf_vectorizer.fit_transform(sent)
      cos.append(cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0])

    context_idx = cos.index(max(cos))

    return context_li[context_idx]

  except:

    return print('해당 내용이 없습니다.')
  # return max(cos)

In [284]:
cos_test(input('질문 : '))

질문 : 폴란드 남부 지역을 병합한 나라는?


'폴란드의 역사에 대해 설명한다 원래 지금의 폴란드 땅에는 게르만족들이 살고 있었다 10세기 경 폴라니에족을 중심으로 나라가 형성되어 민족과 국토의 명칭이 생겨났다 966년 가톨릭을 받아들이며 피아스트 왕조가 성립되었고 그니에즈노와 크라쿠프가 정치적 중심지로 발전했다 200여년의 공국 분할 시대를 거쳐 중앙 집권 국가를 이루었다 1385년 피아스트 왕조에 이어 폴란드리투아니아의 연합 왕조인 야기에오 왕조 14∼16세기 수도 크라쿠프 가 탄생하였다 1410년 그룬발트 전투에서 독일군을 격파 발트해로 통하는 길이 열렸다 16세기에는 유럽의 곡창 지대로 최대 전성기를 맞았다 1573년 야기에오 왕조가 끝나고 귀족들이 국왕을 선출하는 일종의 귀족 공화정이 등장하였고 1596년에 수도를 크라쿠프에서 바르샤바로 이전했다 그러나 선거왕제 選擧王制 의 채용과 바르샤바로 천도한 후 귀족계급의 강대화와 투르크 스웨덴과 전쟁 등으로 국력이 쇠퇴하여 국운이 기울어졌다 18세기 후반에 왕권의 강화와 국가 개혁이 시도되었으나 프로이센 러시아 오스트리아 3국이 점진적으로 폴란드를 침입해 1795년 폴란드를 분할하여 나폴레옹에 의한 바르샤바 공국 시대 18071815 외에는 1795년부터 1918년까지 3국의 지배를 받게 되었다 그의 맞서 1830년 독립을 위한 혁명 정부를 조직 봉기로 이어졌으나 독립투쟁은 실패했다 이후 1차 세계 대전 중 월슨 대통령이 제창한 민족자결주의 원칙에 따라 1918년 독립하였다 19세기의 독립운동의 전통과 제1차 세계 대전을 기회로 국가 재건을 수행한 것이다 그 뒤 바르바니파르토 소장이 지휘하는 군인이 쿠데타를 일으킨다 1939년 나치 독일과 소련의 침략을 받고 서부지역은 나치 독일에 동부는 소련에 분할 점령되었다가 1945년 해방되었다 해방 후 1947년 총선 결과 노동자당의 압승으로 공산당 정부가 수립되었고 냉전을 거치며 소련의 강한 영향력 하에 있었다 그러나 경제 실정과 지도층의 부패로 노동자 파업이 일어났고 1981년 바웬사가 이끄는 자유 노조는 전국

In [413]:
# cos 유사도 추가 - question vs question

def cos_test(user_question):
  token_li = okt.phrases(user_question)
  # token_len = len(token_li) - (len(token_li)-3)
  # token = token_li[:token_len]
  text = ''
  for i in range(len(token_li)):
    text += f'{token_li[i]}|'

  token = text[:-1]

  key_question = test_df.query(f'question.str.contains("{token}")', engine = 'python')
  # key_title = test_df.query(f'title.str.contains("{token}")', engine = 'python')
  # if len(key_title) == 0:
  #   key_question = test_df.query(f'question.str.contains("{token}")', engine = 'python')
  #   if len(key_question) == 0:
  #    key_context = test_df.query(f'context.str.contains("{token}")', engine = 'python')
  #   else:
  #    key_context = key_question.query(f'context.str.contains("{token}")', engine = 'python')
  #    if len(key_context) == 0:
  #     use_context = key_question.drop_duplicates(['context'], ignore_index = True)
  #    else:
  #     use_context = key_context.drop_duplicates(['context'], ignore_index = True)

  # else:
  #   key_question = key_title.query(f'question.str.contains("{token}")', engine = 'python')
  #   if len(key_question) == 0:
  #     key_context = key_title.query(f'context.str.contains("{token}")', engine = 'python')
  #   else:
  #     key_context = key_question.query(f'context.str.contains("{token}")', engine = 'python')
  #     if len(key_context) == 0:
  #       use_context = key_question.drop_duplicates(['context'], ignore_index = True)
  #     else:
  #       use_context = key_context.drop_duplicates(['context'], ignore_index = True)
  use_context = key_question.drop_duplicates(['context'], ignore_index = True)

  context_li = []
  question_li = []
  for i in range(len(use_context)):
    context_li.append(use_context['context'][i])
    question_li.append(use_context['question'][i])

  # context = " ".join(context_li)

  tfidf_vectorizer=TfidfVectorizer()
  cos = []

  for i in range(len(question_li)):
    sent=(question_li[i],
          user_question)
    tfidf_matrix=tfidf_vectorizer.fit_transform(sent)
    cos.append(cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0])

  context_idx = cos.index(max(cos))

  return context_li[context_idx]
  # return question_li[context_idx]
  # return max(cos)

In [323]:
cos_test(input('질문 : '))

질문 : 폴란드 남부 지역을 병합한 나라는?


'폴란드 공화국 폴란드어 rzeczpospolita polska 제치포스폴리타 폴스카 약칭 폴란드 poland 폴란드어 polska 폴스카 은 중앙유럽에 있는 나라이다 폴란드는 16개 주 州 로 나뉘며 수도는 바르샤바이다 인구는 3850만으로언제 유럽 연합 eu 에서 6 번째로 인구가 많은 국가이다 또한 유럽에서 가장 큰 국가 중 하나이기도 한데 면적은 312679 제곱킬로미터이다 주요 종교는 로마 가톨릭이다 북위 46°54˚ 동경 14°24°에 걸쳐 중앙유럽의 대평원 지역에 위치하고 있다 국토의 75가 해발 200m 이하 동쪽으로는 벨라루스 우크라이나 리투아니아 및 러시아 월경지 칼리닌그라드주 남쪽으로는 체코 및 슬로바키아 서쪽으로는 독일 등 7개국과 접하고 있으며 북쪽으로는 발트 해에 접한다 오늘날의 폴란드 지역에는 한때 게르만 부족과 슬라브 부족이 거주했다 폴란드의 역사는 폴란드 공작이 기독교로 개종한 966년에 시작되었다 폴란드의 첫 번째 왕이 1025년에 선출되었고 피아스트 왕조가 부상했다 피아스트 왕조는 1370년에 끝나고 리투아니아와 정치적으로 연합하게 되면서 야기에우워 왕조가 시작되었다 1569년 루블린 조약으로 폴란드리투아니아 연방이 창설되면서 폴란드는 당대 유럽에서 가장 크고 인구가 많은 국가 중 하나로 떠오르며 번영했다 폴란드는 오랫동안 강력한 국가였으나 점차 쇠퇴하면서 결국 1795년 프로이센 러시아 오스트리아에 의해 완전히 분할되었다 폴란드인은 제1차 세계 대전 이후 1918년 11월 11일에 독립을 되찾았다 종교적 관용 때문에 폴란드는 매우 다문화 국가였지만 독일과 긴장된 외교 관계를 유지했다 제2차 세계 대전은 1939년 9월 1일 독일이 폴란드를 침략하면서 시작되었다 이 전쟁으로 폴란드의 유대인 인구를 포함하여 폴란드 국민 600만 명이 사망했다 전쟁이 끝나고 폴란드에 공산주의 정부가 설립되었다 1989년 공산주의의 붕괴 이후 폴란드는 민주주의 공화국이다 폴란드는 지역 강국이다 역동적인 경제를 가지고 있으며 gdp의 ppp는 12 

In [445]:
# question 일치하는 context 추출

def cos_test(user_question):

  key_question = test_df.query(f'question.str.contains("{user_question}")', engine = 'python')
  use_context = key_question.drop_duplicates(['context'], ignore_index = True)
  
  if len(use_context) != 0:
    return use_context['context'][0]
  # context_li = []
  # for i in range(len(use_context)):
  #   context_li.append(use_context['context'][i])

  # # context = " ".join(context_li)

  # tfidf_vectorizer=TfidfVectorizer()
  # cos = []

  # for i in range(len(question_li)):
  #   sent=(question_li[i],
  #         user_question)
  #   tfidf_matrix=tfidf_vectorizer.fit_transform(sent)
  #   cos.append(cosine_similarity(tfidf_matrix[0],tfidf_matrix[1])[0][0])

  # context_idx = cos.index(max(cos))

In [439]:
cos_test(input('질문 : '))

질문 : 폴란드 남부 지역을 병합한 나라는?


'폴란드의 역사에 대해 설명한다 원래 지금의 폴란드 땅에는 게르만족들이 살고 있었다 10세기 경 폴라니에족을 중심으로 나라가 형성되어 민족과 국토의 명칭이 생겨났다 966년 가톨릭을 받아들이며 피아스트 왕조가 성립되었고 그니에즈노와 크라쿠프가 정치적 중심지로 발전했다 200여년의 공국 분할 시대를 거쳐 중앙 집권 국가를 이루었다 1385년 피아스트 왕조에 이어 폴란드리투아니아의 연합 왕조인 야기에오 왕조 14∼16세기 수도 크라쿠프 가 탄생하였다 1410년 그룬발트 전투에서 독일군을 격파 발트해로 통하는 길이 열렸다 16세기에는 유럽의 곡창 지대로 최대 전성기를 맞았다 1573년 야기에오 왕조가 끝나고 귀족들이 국왕을 선출하는 일종의 귀족 공화정이 등장하였고 1596년에 수도를 크라쿠프에서 바르샤바로 이전했다 그러나 선거왕제 選擧王制 의 채용과 바르샤바로 천도한 후 귀족계급의 강대화와 투르크 스웨덴과 전쟁 등으로 국력이 쇠퇴하여 국운이 기울어졌다 18세기 후반에 왕권의 강화와 국가 개혁이 시도되었으나 프로이센 러시아 오스트리아 3국이 점진적으로 폴란드를 침입해 1795년 폴란드를 분할하여 나폴레옹에 의한 바르샤바 공국 시대 18071815 외에는 1795년부터 1918년까지 3국의 지배를 받게 되었다 그의 맞서 1830년 독립을 위한 혁명 정부를 조직 봉기로 이어졌으나 독립투쟁은 실패했다 이후 1차 세계 대전 중 월슨 대통령이 제창한 민족자결주의 원칙에 따라 1918년 독립하였다 19세기의 독립운동의 전통과 제1차 세계 대전을 기회로 국가 재건을 수행한 것이다 그 뒤 바르바니파르토 소장이 지휘하는 군인이 쿠데타를 일으킨다 1939년 나치 독일과 소련의 침략을 받고 서부지역은 나치 독일에 동부는 소련에 분할 점령되었다가 1945년 해방되었다 해방 후 1947년 총선 결과 노동자당의 압승으로 공산당 정부가 수립되었고 냉전을 거치며 소련의 강한 영향력 하에 있었다 그러나 경제 실정과 지도층의 부패로 노동자 파업이 일어났고 1981년 바웬사가 이끄는 자유 노조는 전국